# NB01: Foundation

**Constraint-Based Architectural NCA**

**Version:** 1.2
**Date:** December 2025
**Purpose:** Build and verify all core components before training

---

## Aims

1. Implement 3D Sobel perception filters
2. Implement NCA model architecture (perceive → update → apply)
3. Implement UrbanSceneGenerator with all difficulty levels
4. Implement 3D visualization utilities
5. Verify forward pass works (no training)
6. Verify gradient flow through model

## Success Criteria

- Model produces output of correct shape [B, 8, 32, 32, 32]
- Scene generator produces valid scenes at all difficulty levels
- Gradients flow through all parameters
- Visualization renders correctly

## Changes

**v1.2:**
- **Perception padding**: Changed from zero-padding to REPLICATE padding to eliminate boundary gradient artifacts that caused systematic growth bias toward grid edges
- **Access point randomization**: Access points now placed randomly across valid locations (ground in gap, facades facing gap) instead of only at y ∈ [0, G//3]
- **Axis labels**: Visualization now shows correct tensor dimension mapping (Y=depth, X=left-right, Z=height)

**v1.1:**
- walkable_bias: Changed from -0.5 to 0.1 (allows walkable to grow)

---

## 1. Setup

In [ ]:
# Mount Google Drive for persistence
from google.colab import drive
drive.mount('/content/drive')

# Create project directory structure
import os
PROJECT_ROOT = '/content/drive/MyDrive/Constraint-NCA'
os.makedirs(f'{PROJECT_ROOT}/checkpoints', exist_ok=True)
os.makedirs(f'{PROJECT_ROOT}/figures', exist_ok=True)
os.makedirs(f'{PROJECT_ROOT}/logs', exist_ok=True)
print(f'Project root: {PROJECT_ROOT}')

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from typing import Dict, Tuple, Optional
import json
from datetime import datetime

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

In [ ]:
# Set random seeds for reproducibility
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
print('Random seeds set')

## 2. Configuration

In [ ]:
# Master configuration
CONFIG = {
    # Grid
    'grid_size': 32,           # D = H = W = 32
    'n_channels': 8,           # Total channels
    'n_frozen': 4,             # Frozen channels (existing, ground, access, reserved)
    'n_grown': 4,              # Grown channels (structure, walkable, alive, hidden)

    # Channel indices
    'ch_existing': 0,
    'ch_ground': 1,
    'ch_access': 2,
    'ch_reserved': 3,
    'ch_structure': 4,
    'ch_walkable': 5,
    'ch_alive': 6,
    'ch_hidden': 7,

    # Network
    'hidden_dim': 96,          # Hidden layer size
    'fire_rate': 0.5,          # Stochastic update rate
    'update_scale': 0.15,      # Delta multiplier

    # Initialization
    'xavier_gain': 0.5,        # Xavier init gain
    'structure_bias': 0.1,     # Positive bias for growth
    'walkable_bias': 0.1,      # Positive bias to allow walkable growth (was -0.5)

    # Training (for later notebooks)
    'epochs_per_phase': 400,
    'batch_size': 4,
    'lr_initial': 2e-3,
    'lr_decay': 0.7,
    'grad_clip': 1.0,

    # Perception
    'perception_channels': 32,  # 8 channels * 4 (identity + 3 gradients)
}

print('Configuration loaded')
for k, v in CONFIG.items():
    print(f'  {k}: {v}')

## 3. Perception Module (3D Sobel Filters)

In [ ]:
class Perceive3D(nn.Module):
    """
    3D Sobel perception for Neural Cellular Automata.

    Computes identity + gradient in x, y, z directions for each channel.
    Output: C * 4 channels (identity, grad_x, grad_y, grad_z)
    
    Uses REPLICATE padding to avoid boundary artifacts that cause
    systematic growth bias toward grid edges.
    """

    def __init__(self, n_channels: int = 8):
        super().__init__()
        self.n_channels = n_channels

        # Create 3D Sobel kernels
        # Each kernel is 3x3x3
        sobel_x = self._create_sobel_kernel('x')
        sobel_y = self._create_sobel_kernel('y')
        sobel_z = self._create_sobel_kernel('z')
        identity = self._create_identity_kernel()

        # Stack kernels: [4, 3, 3, 3]
        kernels = torch.stack([identity, sobel_x, sobel_y, sobel_z], dim=0)
        self.register_buffer('kernels', kernels)

    def _create_sobel_kernel(self, direction: str) -> torch.Tensor:
        """
        Create 3D Sobel kernel for gradient in given direction.

        The 3D Sobel kernel is separable:
        - Derivative in target direction: [-1, 0, 1]
        - Smoothing in other directions: [1, 2, 1]
        """
        derivative = torch.tensor([-1., 0., 1.])
        smoothing = torch.tensor([1., 2., 1.])

        if direction == 'x':
            # Gradient in x (width) direction
            kernel = torch.einsum('i,j,k->ijk',
                                  smoothing,  # z
                                  smoothing,  # y
                                  derivative) # x
        elif direction == 'y':
            # Gradient in y (height) direction
            kernel = torch.einsum('i,j,k->ijk',
                                  smoothing,   # z
                                  derivative,  # y
                                  smoothing)   # x
        elif direction == 'z':
            # Gradient in z (depth) direction
            kernel = torch.einsum('i,j,k->ijk',
                                  derivative,  # z
                                  smoothing,   # y
                                  smoothing)   # x
        else:
            raise ValueError(f"Unknown direction: {direction}")

        # Normalize
        kernel = kernel / 16.0
        return kernel

    def _create_identity_kernel(self) -> torch.Tensor:
        """Create identity kernel (center = 1, rest = 0)."""
        kernel = torch.zeros(3, 3, 3)
        kernel[1, 1, 1] = 1.0
        return kernel

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Apply perception to state.

        Args:
            x: State tensor [B, C, D, H, W]

        Returns:
            Perception tensor [B, C*4, D, H, W]
        """
        B, C, D, H, W = x.shape

        # Use REPLICATE padding to avoid boundary artifacts
        # This prevents false gradients at grid edges that cause growth bias
        x_padded = F.pad(x, (1, 1, 1, 1, 1, 1), mode='replicate')

        # Apply each kernel type to all channels
        outputs = []
        for k in range(4):  # identity, grad_x, grad_y, grad_z
            kernel = self.kernels[k:k+1, :, :, :]  # [1, 3, 3, 3]
            kernel = kernel.unsqueeze(0).expand(C, 1, 3, 3, 3)  # [C, 1, 3, 3, 3]

            # Group convolution with no padding (already padded manually)
            out = F.conv3d(x_padded, kernel, padding=0, groups=C)
            outputs.append(out)

        # Concatenate: [B, C*4, D, H, W]
        return torch.cat(outputs, dim=1)


# Test perception module
print('Testing Perceive3D...')
perceive = Perceive3D(n_channels=8).to(device)
test_input = torch.randn(2, 8, 32, 32, 32, device=device)
test_output = perceive(test_input)
print(f'  Input shape:  {test_input.shape}')
print(f'  Output shape: {test_output.shape}')
print(f'  Expected:     [2, 32, 32, 32, 32]')
assert test_output.shape == (2, 32, 32, 32, 32), 'Perception output shape mismatch!'
print('  ✓ Perception module working (with replicate padding)')

## 4. NCA Model Architecture

In [ ]:
class UrbanPavilionNCA(nn.Module):
    """
    Neural Cellular Automaton for urban pavilion generation.

    Architecture:
        Perceive (Sobel) -> Update MLP (1x1x1 conv) -> Apply Delta (gated)

    Channels:
        Frozen [0-3]: existing, ground, access, reserved
        Grown [4-7]:  structure, walkable, alive, hidden
    """

    def __init__(self, config: dict):
        super().__init__()
        self.config = config

        n_channels = config['n_channels']
        hidden_dim = config['hidden_dim']
        perception_dim = n_channels * 4  # identity + 3 gradients
        n_grown = config['n_grown']

        # Perception
        self.perceive = Perceive3D(n_channels)

        # Update network (MLP as 1x1x1 convolutions)
        self.update_net = nn.Sequential(
            nn.Conv3d(perception_dim, hidden_dim, 1),
            nn.ReLU(),
            nn.Conv3d(hidden_dim, hidden_dim, 1),
            nn.ReLU(),
            nn.Conv3d(hidden_dim, n_grown, 1),
        )

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        """Initialize network weights."""
        gain = self.config['xavier_gain']

        for m in self.update_net:
            if isinstance(m, nn.Conv3d):
                nn.init.xavier_uniform_(m.weight, gain=gain)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

        # Set biases for structure and walkable channels
        # Last layer outputs [structure, walkable, alive, hidden]
        last_layer = self.update_net[-1]
        with torch.no_grad():
            last_layer.bias[0] = self.config['structure_bias']  # structure
            last_layer.bias[1] = self.config['walkable_bias']   # walkable

    def forward(self, state: torch.Tensor, steps: int = 1) -> torch.Tensor:
        """
        Run NCA for specified number of steps.

        Args:
            state: Initial state [B, C, D, H, W]
            steps: Number of growth steps

        Returns:
            Final state [B, C, D, H, W]
        """
        for _ in range(steps):
            state = self._step(state)
        return state

    def _step(self, state: torch.Tensor) -> torch.Tensor:
        """
        Single NCA update step.

        1. Perceive neighborhood
        2. Compute update delta
        3. Apply stochastic mask
        4. Gate walkable by structure
        5. Update grown channels only
        """
        B, C, D, H, W = state.shape
        cfg = self.config

        # 1. Perceive
        perception = self.perceive(state)  # [B, C*4, D, H, W]

        # 2. Compute delta
        delta = self.update_net(perception)  # [B, n_grown, D, H, W]

        # 3. Stochastic update mask
        if self.training:
            fire_mask = (torch.rand(B, 1, D, H, W, device=state.device) < cfg['fire_rate']).float()
            delta = delta * fire_mask

        # 4. Gate walkable by structure
        structure = state[:, cfg['ch_structure']:cfg['ch_structure']+1]
        gate = torch.sigmoid(5.0 * (structure - 0.2))  # soft gate

        # Apply gate to walkable delta only (index 1 in delta)
        delta_gated = delta.clone()
        delta_gated[:, 1:2] = delta[:, 1:2] * gate

        # 5. Update grown channels
        new_state = state.clone()
        grown_start = cfg['n_frozen']  # 4
        new_state[:, grown_start:] = state[:, grown_start:] + cfg['update_scale'] * delta_gated

        # Clamp to valid range
        new_state = torch.clamp(new_state, 0.0, 1.0)

        return new_state

    def grow(self, seed: torch.Tensor, steps: int = 50) -> torch.Tensor:
        """
        Grow structure from seed state (inference mode).

        Args:
            seed: Initial state with frozen channels set
            steps: Number of growth steps

        Returns:
            Final grown state
        """
        self.eval()
        with torch.no_grad():
            return self.forward(seed, steps)


# Test NCA model
print('Testing UrbanPavilionNCA...')
model = UrbanPavilionNCA(CONFIG).to(device)

# Count parameters
n_params = sum(p.numel() for p in model.parameters())
print(f'  Parameters: {n_params:,}')

# Test forward pass
test_state = torch.zeros(2, 8, 32, 32, 32, device=device)
test_state[:, 1, 0, :, :] = 1.0  # Ground plane

model.train()
output = model(test_state, steps=5)
print(f'  Input shape:  {test_state.shape}')
print(f'  Output shape: {output.shape}')
assert output.shape == test_state.shape, 'Output shape mismatch!'
print('  ✓ Forward pass working')

# Verify frozen channels unchanged
assert torch.allclose(output[:, :4], test_state[:, :4]), 'Frozen channels modified!'
print('  ✓ Frozen channels preserved')

# Verify grown channels changed
grown_diff = (output[:, 4:] - test_state[:, 4:]).abs().sum()
print(f'  Grown channels change: {grown_diff.item():.4f}')
assert grown_diff > 0, 'Grown channels not updating!'
print('  ✓ Grown channels updating')

## 5. Urban Scene Generator

In [ ]:
class UrbanSceneGenerator:
    """
    Generate urban scenes with buildings and access points.

    Difficulty levels:
        easy:   2 buildings, same height, wide gap
        medium: 2 buildings, different heights, medium gap
        hard:   3+ buildings, varying heights, narrow gap
        random: randomized parameters
    
    Access points are randomized across valid locations:
        - Ground level anywhere in the gap
        - Building facades FACING the gap (not back facades)
        - NOT inside buildings, NOT on rooftops
    """

    def __init__(self, config: dict):
        self.config = config
        self.G = config['grid_size']
        self.C = config['n_channels']

    def generate(self,
                 difficulty: str = 'easy',
                 access_type: str = 'ground',
                 device: str = 'cuda') -> Tuple[torch.Tensor, dict]:
        """
        Generate an urban scene.

        Args:
            difficulty: 'easy', 'medium', 'hard', 'random'
            access_type: 'ground', 'facade', 'mixed'
            device: torch device

        Returns:
            state: [1, C, D, H, W] tensor with frozen channels set
            metadata: dict with scene information
        """
        G = self.G
        state = torch.zeros(1, self.C, G, G, G, device=device)

        # Set ground plane (z=0)
        state[:, self.config['ch_ground'], 0, :, :] = 1.0

        # Get difficulty parameters
        params = self._get_difficulty_params(difficulty)

        # Place buildings
        building_info = self._place_buildings(state, params)

        # Place access points (randomized across valid locations)
        access_info = self._place_access_points(state, access_type, params, building_info)

        metadata = {
            'difficulty': difficulty,
            'access_type': access_type,
            'buildings': building_info,
            'access_points': access_info,
            'gap_width': params['gap_width'],
        }

        return state, metadata

    def _get_difficulty_params(self, difficulty: str) -> dict:
        """Get parameters for difficulty level."""
        G = self.G

        if difficulty == 'easy':
            return {
                'n_buildings': 2,
                'height_range': (12, 16),
                'height_variance': False,
                'width_range': (8, 12),
                'gap_width': random.randint(12, 16),
                'n_access': random.randint(2, 3),
            }
        elif difficulty == 'medium':
            return {
                'n_buildings': 2,
                'height_range': (10, 20),
                'height_variance': True,
                'width_range': (6, 10),
                'gap_width': random.randint(8, 12),
                'n_access': random.randint(2, 4),
            }
        elif difficulty == 'hard':
            return {
                'n_buildings': random.randint(2, 4),
                'height_range': (8, 24),
                'height_variance': True,
                'width_range': (5, 8),
                'gap_width': random.randint(5, 8),
                'n_access': random.randint(4, 6),
            }
        else:  # random
            return {
                'n_buildings': random.randint(2, 4),
                'height_range': (8, 24),
                'height_variance': True,
                'width_range': (5, 12),
                'gap_width': random.randint(5, 16),
                'n_access': random.randint(2, 6),
            }

    def _place_buildings(self, state: torch.Tensor, params: dict) -> list:
        """Place buildings in the scene."""
        G = self.G
        ch = self.config['ch_existing']
        buildings = []

        n_buildings = params['n_buildings']
        gap_width = params['gap_width']
        gap_center = G // 2

        if n_buildings == 2:
            # Building 1 (left side)
            w1 = random.randint(*params['width_range'])
            d1 = random.randint(G//2, G-2)
            h1 = random.randint(*params['height_range'])
            x1_end = gap_center - gap_width // 2
            x1_start = max(0, x1_end - w1)

            # Randomize y position slightly
            y1_start = random.randint(0, G//6)
            y1_end = min(G, y1_start + d1)

            state[:, ch, :h1, y1_start:y1_end, x1_start:x1_end] = 1.0
            buildings.append({
                'x': (x1_start, x1_end), 
                'y': (y1_start, y1_end), 
                'z': (0, h1),
                'gap_facing_x': x1_end  # Facade facing the gap
            })

            # Building 2 (right side)
            w2 = random.randint(*params['width_range'])
            d2 = random.randint(G//2, G-2)
            h2 = h1 if not params['height_variance'] else random.randint(*params['height_range'])
            x2_start = gap_center + gap_width // 2
            x2_end = min(G, x2_start + w2)

            # Randomize y position slightly  
            y2_start = random.randint(0, G//6)
            y2_end = min(G, y2_start + d2)

            state[:, ch, :h2, y2_start:y2_end, x2_start:x2_end] = 1.0
            buildings.append({
                'x': (x2_start, x2_end), 
                'y': (y2_start, y2_end), 
                'z': (0, h2),
                'gap_facing_x': x2_start  # Facade facing the gap
            })

        else:
            # Multiple buildings: distribute around the perimeter
            for i in range(n_buildings):
                w = random.randint(*params['width_range'])
                d = random.randint(4, 8)
                h = random.randint(*params['height_range'])

                if i % 4 == 0:  # Left
                    x_start, x_end = 0, w
                    y_start = random.randint(0, G - d)
                    y_end = y_start + d
                    gap_facing = x_end
                elif i % 4 == 1:  # Right
                    x_start, x_end = G - w, G
                    y_start = random.randint(0, G - d)
                    y_end = y_start + d
                    gap_facing = x_start
                elif i % 4 == 2:  # Back
                    y_start, y_end = G - d, G
                    x_start = random.randint(0, G - w)
                    x_end = x_start + w
                    gap_facing = None  # Back building, no gap-facing facade
                else:  # Front corners
                    y_start, y_end = 0, d
                    x_start = 0 if random.random() < 0.5 else G - w
                    x_end = x_start + w
                    gap_facing = x_end if x_start == 0 else x_start

                state[:, ch, :h, y_start:y_end, x_start:x_end] = 1.0
                buildings.append({
                    'x': (x_start, x_end), 
                    'y': (y_start, y_end), 
                    'z': (0, h),
                    'gap_facing_x': gap_facing
                })

        return buildings

    def _place_access_points(self, state: torch.Tensor, access_type: str,
                             params: dict, buildings: list) -> list:
        """
        Place access points in valid locations:
        - Ground level anywhere in the gap (not inside buildings)
        - Building facades FACING the gap (not back facades, not rooftops)
        """
        G = self.G
        ch = self.config['ch_access']
        n_access = params['n_access']
        access_points = []
        
        # Determine gap region (between buildings)
        gap_x_start = 0
        gap_x_end = G
        for bldg in buildings:
            bx_start, bx_end = bldg['x']
            if bx_end <= G // 2:  # Left building
                gap_x_start = max(gap_x_start, bx_end)
            else:  # Right building
                gap_x_end = min(gap_x_end, bx_start)

        for i in range(n_access):
            # Randomly choose access type for this point
            if access_type == 'ground':
                place_type = 'ground'
            elif access_type == 'facade':
                place_type = 'facade'
            else:  # mixed
                place_type = random.choice(['ground', 'facade'])
            
            if place_type == 'ground':
                # Ground access: anywhere in the gap, randomized y position
                x = random.randint(gap_x_start + 1, gap_x_end - 2)
                y = random.randint(0, G - 3)  # Full y range
                z = 0
                
                # Mark 2x2 access area
                state[:, ch, z:z+2, y:y+2, x:x+2] = 1.0
                access_points.append({'x': x, 'y': y, 'z': z, 'type': 'ground'})
                
            else:  # facade
                # Facade access: on building facades FACING the gap
                # Filter buildings that have gap-facing facades
                valid_buildings = [b for b in buildings if b.get('gap_facing_x') is not None]
                
                if len(valid_buildings) > 0:
                    bldg = random.choice(valid_buildings)
                    bx_start, bx_end = bldg['x']
                    by_start, by_end = bldg['y']
                    bz_max = bldg['z'][1]
                    gap_x = bldg['gap_facing_x']
                    
                    # Place on the gap-facing facade at random height (not rooftop)
                    z = random.randint(1, max(2, bz_max - 2))  # Above ground, below roof
                    y = random.randint(by_start, max(by_start + 1, by_end - 2))
                    
                    # X position: just outside the building on gap-facing side
                    if gap_x == bx_end:  # Left building, access on right edge
                        x = bx_end
                    else:  # Right building, access on left edge
                        x = max(0, bx_start - 1)
                    
                    # Ensure within bounds
                    x = max(0, min(G - 2, x))
                    y = max(0, min(G - 2, y))
                    z = max(1, min(G - 2, z))
                    
                    state[:, ch, z:z+2, y:y+2, x:x+2] = 1.0
                    access_points.append({'x': x, 'y': y, 'z': z, 'type': 'facade'})
                else:
                    # Fallback to ground if no valid facades
                    x = random.randint(gap_x_start + 1, gap_x_end - 2)
                    y = random.randint(0, G - 3)
                    z = 0
                    state[:, ch, z:z+2, y:y+2, x:x+2] = 1.0
                    access_points.append({'x': x, 'y': y, 'z': z, 'type': 'ground'})

        return access_points

    def batch(self, difficulty: str, access_type: str,
              batch_size: int, device: str) -> torch.Tensor:
        """Generate a batch of scenes."""
        scenes = []
        for _ in range(batch_size):
            scene, _ = self.generate(difficulty, access_type, device)
            scenes.append(scene)
        return torch.cat(scenes, dim=0)


# Test scene generator
print('Testing UrbanSceneGenerator...')
scene_gen = UrbanSceneGenerator(CONFIG)

for difficulty in ['easy', 'medium', 'hard']:
    scene, meta = scene_gen.generate(difficulty, 'mixed', device)
    print(f'  {difficulty}: shape={scene.shape}, buildings={len(meta["buildings"])}, '
          f'access={len(meta["access_points"])}, gap={meta["gap_width"]}')
    for ap in meta['access_points']:
        print(f'    - {ap["type"]}: x={ap["x"]}, y={ap["y"]}, z={ap["z"]}')

# Test batch generation
batch = scene_gen.batch('medium', 'ground', 4, device)
print(f'  Batch: {batch.shape}')
assert batch.shape == (4, 8, 32, 32, 32), 'Batch shape mismatch!'
print('  ✓ Scene generator working (with randomized access points)')

## 6. Visualization Utilities

In [ ]:
def visualize_scene(state: torch.Tensor,
                    title: str = 'Urban Scene',
                    show_structure: bool = True,
                    show_walkable: bool = False,
                    threshold: float = 0.5,
                    figsize: Tuple[int, int] = (12, 5)):
    """
    Visualize a voxel scene with buildings, ground, access, and structure.
    
    Note on axes: The tensor has shape [C, Z, Y, X] where:
        - Z is vertical (height)
        - Y is depth (front to back)
        - X is horizontal (left to right)
    
    After transpose(1, 2, 0), matplotlib displays:
        - Plot X-axis = Tensor Y (depth)
        - Plot Y-axis = Tensor X (horizontal)
        - Plot Z-axis = Tensor Z (height)
    """
    # Handle batch dimension
    if state.dim() == 5:
        state = state[0]

    state = state.cpu().numpy()

    fig = plt.figure(figsize=figsize)

    # 3D view
    ax1 = fig.add_subplot(121, projection='3d')

    # Extract channels
    existing = state[CONFIG['ch_existing']] > 0.5
    ground = state[CONFIG['ch_ground']] > 0.5
    access = state[CONFIG['ch_access']] > 0.5
    structure = state[CONFIG['ch_structure']] > threshold if show_structure else None
    walkable = state[CONFIG['ch_walkable']] > threshold if show_walkable else None

    # Plot buildings (gray)
    if existing.any():
        ax1.voxels(existing.transpose(1, 2, 0), facecolors='gray', alpha=0.5, edgecolor='darkgray', linewidth=0.1)

    # Plot access points (green)
    if access.any():
        ax1.voxels(access.transpose(1, 2, 0), facecolors='green', alpha=0.9, edgecolor='darkgreen', linewidth=0.1)

    # Plot structure (blue)
    if structure is not None and structure.any():
        ax1.voxels(structure.transpose(1, 2, 0), facecolors='royalblue', alpha=0.7, edgecolor='navy', linewidth=0.1)

    # Plot walkable (orange)
    if walkable is not None and walkable.any():
        ax1.voxels(walkable.transpose(1, 2, 0), facecolors='orange', alpha=0.6, edgecolor='darkorange', linewidth=0.1)

    # Clear axis labels showing tensor dimension mapping
    ax1.set_xlabel('Y (depth)')
    ax1.set_ylabel('X (left-right)')
    ax1.set_zlabel('Z (height)')
    ax1.set_title(f'{title} (3D)')

    # Plan view (top-down)
    ax2 = fig.add_subplot(122)

    # Create composite image
    G = state.shape[1]
    plan = np.zeros((G, G, 3))

    # Buildings in gray
    building_plan = existing.max(axis=0)  # max along Z
    plan[building_plan] = [0.5, 0.5, 0.5]

    # Structure in blue
    if structure is not None:
        struct_plan = structure.max(axis=0)
        plan[struct_plan] = [0.2, 0.4, 0.8]

    # Access in green
    access_plan = access.max(axis=0)
    plan[access_plan] = [0.2, 0.8, 0.2]

    ax2.imshow(plan.transpose(1, 0, 2), origin='lower')
    ax2.set_xlabel('Y (depth)')
    ax2.set_ylabel('X (left-right)')
    ax2.set_title(f'{title} (Plan)')

    plt.tight_layout()
    plt.show()


def visualize_growth(model: nn.Module,
                     seed: torch.Tensor,
                     steps: list = [0, 10, 25, 50],
                     figsize: Tuple[int, int] = (16, 4)):
    """
    Visualize NCA growth at different time steps.
    """
    model.eval()
    states = [seed.clone()]

    with torch.no_grad():
        state = seed.clone()
        for s in range(max(steps)):
            state = model(state, steps=1)
            if (s + 1) in steps:
                states.append(state.clone())

    fig, axes = plt.subplots(1, len(steps), figsize=figsize, subplot_kw={'projection': '3d'})

    for ax, step, state in zip(axes, steps, states):
        s = state[0].cpu().numpy()

        existing = s[CONFIG['ch_existing']] > 0.5
        structure = s[CONFIG['ch_structure']] > 0.3

        if existing.any():
            ax.voxels(existing.transpose(1, 2, 0), facecolors='gray', alpha=0.3)
        if structure.any():
            ax.voxels(structure.transpose(1, 2, 0), facecolors='royalblue', alpha=0.7)

        ax.set_title(f'Step {step}')
        ax.set_xlabel('Y (depth)')
        ax.set_ylabel('X (left-right)')
        ax.set_zlabel('Z (height)')

    plt.tight_layout()
    plt.show()


# Test visualization
print('Testing visualization...')
scene, meta = scene_gen.generate('easy', 'ground', device)
visualize_scene(scene, f'Easy Scene (gap={meta["gap_width"]})')

# Test growth visualization
print('Testing growth visualization...')
visualize_growth(model, scene, steps=[0, 5, 15, 30])
print('  ✓ Visualization working')

## 7. Gradient Flow Verification

In [ ]:
def verify_gradient_flow(model: nn.Module,
                         state: torch.Tensor,
                         steps: int = 10) -> dict:
    """
    Verify gradients flow through all model parameters.

    Args:
        model: NCA model
        state: Input state
        steps: Number of forward steps

    Returns:
        Dictionary of gradient statistics per layer
    """
    model.train()
    model.zero_grad()

    # Forward pass
    output = model(state, steps=steps)

    # Simple loss: sum of structure channel
    loss = output[:, CONFIG['ch_structure']].sum()

    # Backward pass
    loss.backward()

    # Check gradients
    grad_stats = {}
    for name, param in model.named_parameters():
        if param.grad is not None:
            grad = param.grad
            grad_stats[name] = {
                'shape': list(param.shape),
                'grad_norm': grad.norm().item(),
                'grad_mean': grad.abs().mean().item(),
                'grad_max': grad.abs().max().item(),
                'has_grad': True,
            }
        else:
            grad_stats[name] = {
                'shape': list(param.shape),
                'has_grad': False,
            }

    return grad_stats


# Test gradient flow
print('Verifying gradient flow...')
scene, _ = scene_gen.generate('easy', 'ground', device)
scene.requires_grad = False  # Frozen channels don't need grad

grad_stats = verify_gradient_flow(model, scene, steps=10)

all_have_grad = True
print('\nGradient statistics:')
print('-' * 70)
for name, stats in grad_stats.items():
    if stats['has_grad']:
        print(f'{name:40} | norm={stats["grad_norm"]:.2e} | mean={stats["grad_mean"]:.2e}')
        if stats['grad_norm'] == 0:
            print(f'  WARNING: Zero gradient!')
            all_have_grad = False
    else:
        print(f'{name:40} | NO GRADIENT')
        all_have_grad = False

print('-' * 70)
if all_have_grad:
    print('✓ All parameters have gradients')
else:
    print('✗ Some parameters missing gradients!')

## 8. Save Foundation Components

In [ ]:
# Save configuration
config_path = f'{PROJECT_ROOT}/config.json'
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)
print(f'Configuration saved to {config_path}')

# Save model architecture info
model_info = {
    'n_parameters': sum(p.numel() for p in model.parameters()),
    'architecture': str(model),
    'config': CONFIG,
    'timestamp': datetime.now().isoformat(),
}

info_path = f'{PROJECT_ROOT}/model_info.json'
with open(info_path, 'w') as f:
    json.dump(model_info, f, indent=2, default=str)
print(f'Model info saved to {info_path}')

print('\n' + '='*60)
print('FOUNDATION COMPLETE')
print('='*60)
print('\nAll components verified:')
print('  ✓ Perceive3D (Sobel filters)')
print('  ✓ UrbanPavilionNCA (model)')
print('  ✓ UrbanSceneGenerator')
print('  ✓ Visualization utilities')
print('  ✓ Gradient flow')
print('\nReady for NB02_Phase1_Structural')

---

## Summary

### What was implemented:

1. **Perceive3D**: 3D Sobel perception with identity + gradients (8 → 32 channels)
2. **UrbanPavilionNCA**: Full NCA with perceive → update → apply, structure-gated walkable
3. **UrbanSceneGenerator**: Procedural urban scenes at easy/medium/hard difficulty
4. **Visualization**: 3D voxel plots and plan views
5. **Gradient verification**: Confirmed gradients flow through all parameters

### Next steps (NB02):

1. Implement ConnectivityLoss and CantileverLoss
2. Implement training loop
3. Train Phase 1: Structural Soundness

---

*NB01_Foundation_v1.0 - December 2025*